# Entrenamiento de YOLO11 en Google Colab

Este cuaderno entrena un modelo YOLO11 con el dataset personalizado ubicado originalmente en `tpfinal-v3/services/worker-ai/finetuning/dataset`. Ejecuta las celdas en orden y ajusta los hiperparámetros según tus necesidades.


## 0. Antes de empezar

1. Comprime la carpeta `dataset` en un archivo `.zip` (incluyendo `data.yaml`, `images/`, `labels/`).
2. Abre este cuaderno en Google Colab y asegúrate de que la sesión tenga acceso a GPU (Menú `Entorno de ejecución > Cambiar tipo de entorno > GPU`).
3. Ejecuta cada celda secuencialmente (Ctrl/Cmd + Intro). Si modificas alguna celda, vuelve a ejecutarla antes de continuar.


In [ ]:
!nvidia-smi


In [ ]:
# Instalación de dependencias principales (Ultralytics incluye YOLOv8/YOLO11)
!pip install -q ultralytics==8.3.40


## 1. Preparar el dataset

Sube el archivo `.zip` del dataset cuando se solicite. El script localizará automáticamente el `data.yaml` y ajustará la ruta para el entorno de Colab.


In [ ]:
#@title Montar Google Drive (opcional)
# Si prefieres trabajar desde Google Drive, ejecuta esta celda y sube el dataset allí.
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
import zipfile
from pathlib import Path
from google.colab import files
import yaml

DATASETS_DIR = Path("/content/datasets")
DATASETS_DIR.mkdir(parents=True, exist_ok=True)

print("Selecciona el archivo .zip del dataset (por ejemplo, dataset.zip)...")
uploaded = files.upload()

zip_names = [name for name in uploaded if name.lower().endswith(".zip")]
if not zip_names:
    raise ValueError("Debes subir al menos un archivo .zip con el dataset en formato YOLO.")

dataset_zip_local = Path(zip_names[0])
dataset_zip_path = DATASETS_DIR / dataset_zip_local.name
if dataset_zip_path.exists():
    dataset_zip_path.unlink()
dataset_zip_local.rename(dataset_zip_path)

print(f"Descomprimiendo {dataset_zip_path}...")
with zipfile.ZipFile(dataset_zip_path, "r") as zip_ref:
    zip_ref.extractall(DATASETS_DIR)

data_yaml_candidates = sorted(DATASETS_DIR.rglob("data.yaml"))
if not data_yaml_candidates:
    raise FileNotFoundError("No se encontró data.yaml dentro del zip. Verifica la estructura.")

data_yaml_path = data_yaml_candidates[0]
dataset_root = data_yaml_path.parent.resolve()

with data_yaml_path.open("r") as f:
    data_config = yaml.safe_load(f)

data_config["path"] = str(dataset_root)
with data_yaml_path.open("w") as f:
    yaml.safe_dump(data_config, f, sort_keys=False)

names = list(data_config["names"].values()) if isinstance(data_config["names"], dict) else data_config["names"]
print(f"Dataset listo en: {dataset_root}")
print(f"Archivo data.yaml actualizado: {data_yaml_path}")
print(f"Clases detectadas ({len(names)}): {names}")


## 2. Definir hiperparámetros de entrenamiento

Ajusta estos valores según la disponibilidad de GPU, el tamaño del dataset y los objetivos del modelo.


In [ ]:
from pathlib import Path

MODEL_VARIANT = "yolo11n.pt"  # Cambia a yolo11s.pt / yolo11m.pt si necesitas más capacidad
EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = 640
PATIENCE = 20  # Detiene el entrenamiento si no mejora en PATIENCE épocas
DEVICE = 0  # 0 = GPU. Usa 'cpu' si no hay GPU disponible
RUN_NAME = "yolo11_custom"
PROJECT_DIR = "runs/train"

print(f"Modelo base: {MODEL_VARIANT}")
print(f"Épocas: {EPOCHS}, Batch size: {BATCH_SIZE}, Imagen: {IMG_SIZE}px")
print(f"Los resultados se guardarán en: {PROJECT_DIR}/{RUN_NAME}")


## 3. Entrenar el modelo

Esta celda inicia el entrenamiento usando Ultralytics. El mejor checkpoint quedará en `runs/train/<RUN_NAME>/weights/best.pt`.


In [ ]:
from ultralytics import YOLO

if 'data_yaml_path' not in globals():
    raise RuntimeError("Ejecuta primero la celda de carga del dataset para definir data_yaml_path.")

model = YOLO(MODEL_VARIANT)
train_results = model.train(
    data=str(data_yaml_path),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    project=PROJECT_DIR,
    name=RUN_NAME,
    device=DEVICE,
    pretrained=True,
    verbose=True,
)

best_model_path = Path(model.trainer.best).resolve()
print(f"Entrenamiento finalizado. Mejor checkpoint: {best_model_path}")


## 4. Evaluar el modelo entrenado


In [ ]:
val_results = model.val(
    data=str(data_yaml_path),
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
)

print("Métricas de validación:")
for metric, value in val_results.results_dict.items():
    print(f"  {metric}: {value:.4f}")


In [ ]:
from IPython.display import Image, display

results_plot = Path(PROJECT_DIR) / RUN_NAME / "results.png"
if results_plot.exists():
    display(Image(filename=results_plot))
else:
    print("results.png no se generó todavía. Revisa que el entrenamiento haya finalizado correctamente.")


## 5. Exportar y descargar artefactos (opcional)

Exporta el modelo a ONNX y descarga los archivos resultantes a tu PC.


In [ ]:
if 'best_model_path' not in globals():
    raise RuntimeError("Ejecuta primero la celda de entrenamiento para definir best_model_path.")

best_model = YOLO(str(best_model_path))
export_path = best_model.export(format="onnx", imgsz=IMG_SIZE, opset=12, simplify=True)
print(f"Modelo exportado a ONNX: {export_path}")


In [ ]:
import shutil
from google.colab import files

artifacts_dir = best_model_path.parent
zip_name = f"{RUN_NAME}_artefactos"
zip_path = shutil.make_archive(zip_name, "zip", root_dir=artifacts_dir)
print(f"Artefactos comprimidos en: {zip_path}")
files.download(zip_path)
